In [3]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys
import warnings
# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import os
import pandas as pd

SUB = "/kaggle/input/subescobangla-speech-emotion-dataset/SUBESCO"
fname = os.listdir(SUB)
fname.sort()

emotion_mapping = {
    'SAD': 'sad',
    'NEUTRAL': 'neutral',
    'ANGRY': 'angry',
    'DISGUST': 'disgust',
    'FEAR': 'fear',
    'SURPRISE': 'surprise',
    'HAPPY': 'happy'
}

emotion = []
gender = []
path = []

for f in fname:
    parts = f.replace('.wav', '').split('_')
    # Emotion is second to last part of the split filename
    file_emotion = parts[-2]
    file_emotion = emotion_mapping.get(file_emotion, 'unknown')  # Emotion keys are already uppercase

    # Gender is indicated by the first character, 'M' for Male, 'F' for Female
    file_gender = 'male' if parts[0] == 'M' else 'female'

    emotion.append(file_emotion)
    gender.append(file_gender)
    path.append(os.path.join(SUB, f))

# Create a DataFrame
SUBESCO_df = pd.DataFrame({
    'labels': [f"{g}_{e}" for g, e in zip(gender, emotion)],
    'source': 'SUBESCO',
    'path': path
})

# Save the DataFrame to a CSV file
csv_file_path = '/kaggle/working/SUBESCO_data.csv'
SUBESCO_df.to_csv(csv_file_path, index=False)

# If you want to print out the first few rows of the DataFrame
print(SUBESCO_df.head())

# To print the label counts
print(SUBESCO_df['labels'].value_counts())


         labels   source                                               path
0  female_angry  SUBESCO  /kaggle/input/subescobangla-speech-emotion-dat...
1  female_angry  SUBESCO  /kaggle/input/subescobangla-speech-emotion-dat...
2  female_angry  SUBESCO  /kaggle/input/subescobangla-speech-emotion-dat...
3  female_angry  SUBESCO  /kaggle/input/subescobangla-speech-emotion-dat...
4  female_angry  SUBESCO  /kaggle/input/subescobangla-speech-emotion-dat...
labels
female_angry       500
female_disgust     500
female_fear        500
female_happy       500
female_neutral     500
female_sad         500
female_surprise    500
male_angry         500
male_disgust       500
male_fear          500
male_happy         500
male_neutral       500
male_sad           500
male_surprise      500
Name: count, dtype: int64


In [6]:
print(SUBESCO_df)

             labels   source  \
0      female_angry  SUBESCO   
1      female_angry  SUBESCO   
2      female_angry  SUBESCO   
3      female_angry  SUBESCO   
4      female_angry  SUBESCO   
...             ...      ...   
6995  male_surprise  SUBESCO   
6996  male_surprise  SUBESCO   
6997  male_surprise  SUBESCO   
6998  male_surprise  SUBESCO   
6999  male_surprise  SUBESCO   

                                                   path  
0     /kaggle/input/subescobangla-speech-emotion-dat...  
1     /kaggle/input/subescobangla-speech-emotion-dat...  
2     /kaggle/input/subescobangla-speech-emotion-dat...  
3     /kaggle/input/subescobangla-speech-emotion-dat...  
4     /kaggle/input/subescobangla-speech-emotion-dat...  
...                                                 ...  
6995  /kaggle/input/subescobangla-speech-emotion-dat...  
6996  /kaggle/input/subescobangla-speech-emotion-dat...  
6997  /kaggle/input/subescobangla-speech-emotion-dat...  
6998  /kaggle/input/subescobangla-s

In [7]:
pip install resampy

  Obtaining dependency information for resampy from https://files.pythonhosted.org/packages/4d/b9/3b00ac340a1aab3389ebcc52c779914a44aadf7b0cb7a3bf053195735607/resampy-0.4.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.6 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Importing required libraries
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other

import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob
import os
import pickle
import IPython.display as ipd  # To play sound in the notebook

In [12]:
ref = pd.read_csv("/kaggle/working/SUBESCO_data.csv")
ref.head()

,labels,source,path
0,female_angry,SUBESCO,/kaggle/input/subescobangla-speech-emotion-dat...
1,female_angry,SUBESCO,/kaggle/input/subescobangla-speech-emotion-dat...
2,female_angry,SUBESCO,/kaggle/input/subescobangla-speech-emotion-dat...
3,female_angry,SUBESCO,/kaggle/input/subescobangla-speech-emotion-dat...
4,female_angry,SUBESCO,/kaggle/input/subescobangla-speech-emotion-dat...


In [13]:
import os
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms


# Define the transformation for ResNet18
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Directory to save spectrogram images
spectrogram_dir = '/kaggle/working/spectogram'
os.makedirs(spectrogram_dir, exist_ok=True)

# Function to convert audio to spectrogram image and save
def audio_to_spectrogram(audio_path, label, save_dir, transform):
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)

    plt.figure(figsize=(3.5, 3.5))
    librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
    plt.axis('off')
    # Use a temporary file path for the spectrogram
    temp_spec_path = 'temp_spectrogram.png'
    plt.savefig(temp_spec_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Open the image and apply any necessary transformations
    image = Image.open(temp_spec_path).convert('RGB')

    # Save the spectrogram image with a label prefix
    image_save_path = os.path.join(save_dir, f"{label}_{os.path.basename(audio_path)}.png")
    image.save(image_save_path)  # Save the PIL image before it becomes a tensor

    # Now apply the transform and continue with your process
    if transform:
        image = transform(image)  # image is now a tensor
count=0
# Iterate over the DataFrame and process each audio file
for _, row in ref.iterrows():
    audio_path = row['path']
    label = row['labels']
    audio_to_spectrogram(audio_path, label, spectrogram_dir, transform)
    count+=1



In [14]:
import os
import torch
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MelSpectrogramDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.files = [f for f in os.listdir(root_dir) if f.endswith('.png')]
        
        # Extract labels and ensure they are unique
        self.labels = [self._extract_label(f) for f in self.files]
        self.classes = sorted(set(self.labels))
        
        # Create a mapping from label strings to integers
        self.label_to_index = {label: idx for idx, label in enumerate(self.classes)}

    def _extract_label(self, filename):
        # Customize this method based on how your labels are encoded in the filenames
        parts = filename.split('_')
        label = '_'.join(parts[:2])  # Assuming the label is in the first two parts
        return label
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.files[idx])
        image = read_image(img_name)
        label_str = self._extract_label(self.files[idx])
        label = self.label_to_index[label_str]  # Convert label to index

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations for the input data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set the directory containing your Mel spectrograms
mel_spectrogram_dir = '/kaggle/working/spectogram'
# Create the dataset and dataloader
mel_dataset = MelSpectrogramDataset(root_dir=mel_spectrogram_dir, transform=transform)
mel_loader = DataLoader(mel_dataset, batch_size=32, shuffle=True)

In [15]:
from torch.utils.data import random_split

# Define the proportion of the dataset to use for training
train_proportion = 0.8

# Calculate the number of samples for train and test sets
num_train = int(len(mel_dataset) * train_proportion)
num_test = len(mel_dataset) - num_train

# Split the dataset into train and test sets
train_dataset, test_dataset = random_split(mel_dataset, [num_train, num_test])

# Create DataLoaders for train and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [10]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [17]:
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
# Initialize the model

original_dataset = mel_dataset

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

# Use the classes from the original ImageFolder dataset
model.fc = nn.Linear(num_ftrs, len(original_dataset.classes))

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Training loop
num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {100 * correct / total}%')

print('Training complete')



Epoch [1/10], Validation Accuracy: 70.42857142857143%
Epoch [2/10], Validation Accuracy: 75.42857142857143%
Epoch [3/10], Validation Accuracy: 81.64285714285714%
Epoch [4/10], Validation Accuracy: 83.71428571428571%
Epoch [5/10], Validation Accuracy: 84.14285714285714%
Epoch [6/10], Validation Accuracy: 81.42857142857143%
Epoch [7/10], Validation Accuracy: 81.28571428571429%
Epoch [8/10], Validation Accuracy: 85.35714285714286%
Epoch [9/10], Validation Accuracy: 84.85714285714286%
Epoch [10/10], Validation Accuracy: 89.42857142857143%
Training complete
